In [13]:
import numpy as np
import pandas as pd
import string
import spacy
import nltk
import matplotlib.pyplot as plt
from collections import Counter
import seaborn as sns

### MISC functions

In [14]:
def syllable_count(word):
    word = word.lower()
    count = 0
    vowels = "aeiouy"
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
    if word.endswith("e"):
        count -= 1
    if count == 0:
        count += 1
    return count

In [15]:
def check_consonant(x): 
      
    # convert string in lowercase
    x = x.lower() 
  
    return not (x == 'a' or x == 'e' or 
                x == 'i' or x == 'o' or 
                x == 'u') 

# function to count consonants  
def countConsonants(string): 
      
    count = 0
      
    for i in range(len(string)): 
  
        # To check is character is Consonant 
        if (check_consonant(string[i])): 
            count += 1
              
    return count 

In [16]:
def remove_noise(text):
    x=text.split()
    if(len(x)==1):
        new = text.replace('"','').replace(':','').replace('“','').replace('”','').replace('-',' ').replace('—','').replace(')','').replace('(','').replace(".","").replace(",","").replace("€","").replace("$","").replace("£","").replace("—","") .replace("’","").replace("→","").replace("%","").replace(";","").replace("]","").replace("[","").replace("/","")
        return new
    new_text=[]
    for word in x:
        new_text.append(word.replace('"','').replace(':','').replace('“','').replace('”','').replace('-',' ').replace('—','').replace(')','').replace('(','').replace(".","").replace(",","").replace("€","").replace("$","").replace("£","").replace("—","") .replace("’","").replace("→","").replace("%","").replace(";","").replace("]","").replace("[","").replace("/",""))

    return " ".join(new_text)

In [17]:
def flatten(t):
    return [item for sublist in t for item in sublist]

In [18]:
def word_gst(matrix, word):
    for pair in matrix:
        if pair[0] == word:
            return pair[1]

In [19]:
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 2:
            result.append(token)

    return result


### Main Functions
Every sentecnce is extracted from the input file and is added to a new dataframe; also total complexity of that sentence, all the words concatenanted in a string and all the gold standards concatenanted in a string are appended to the new transformed dataframe.

In [ ]:
def transform_df(df, train_flag):
    if train_flag:
        df.drop(df.loc[df['offset_start']==1].index, inplace=True) #drop rows with offset start at 1
    
    #columns
    sentences = []
    total_gs = [] #gs=gold standard
    all_tw = []   #tw=target words
    all_gs = []

    #accumulators
    tw =[]
    gold_std = []
    gs_acc = 0
    temp = df['sentence'][df.first_valid_index()]
    nr = 0
    count = 0
    for idx in df.index:
        if temp != df['sentence'][idx]:
            sentences.append(temp)
            all_tw.append("|".join(map(str, tw))) #create a string from the list
            
            #if train_flag:
            total_gs.append(gs_acc)  
            all_gs.append(" ".join(map(str, gold_std)))
            gold_std.clear()
            gs_acc = 0
                
            tw.clear()
            temp = df['sentence'][idx]

        tw.append(df['word'][idx])
        
        gs_acc += df['gold_std'][idx]
        gold_std.append(df['gold_std'][idx])


    sentences.append(temp)
    all_tw.append("|".join(map(str, tw))) #create a string from the list
    #print(all_tw)

    #if train_flag:
    total_gs.append(gs_acc)  
    all_gs.append(" ".join(map(str, gold_std)))
#if train_flag:
    #print(all_tw)
    return pd.DataFrame(list(zip(sentences, total_gs, all_tw, all_gs)), columns = ['sentence', 'gold_std', 'target_words', 'gold_stds']) #the new columns for the dataframe
    #else:
        #return pd.DataFrame(list(zip(sentences, all_tw)), columns = ['sentence', 'target_words'])

In [ ]:
def extract_features(input_file, train_flag):
    nlp = spacy.load('en_core_web_sm')
    twords_lem = [] #all the target words processed and lemmatized, ready to be matched with the dictionary based on he sentences
    eligible_sentences = []
    twords_gstd_all = [] #matrix of processed target words with their gold standard
    upd_tw = [] #target_words
    upd_gs = [] #target words gold standard
    for idx,row in input_file.iterrows(): # for every row from the grouped df
        temp = row[0]

        tokenized_sentence = nlp(row[0])#remove noise from the sentence
        #decode words and gold standard
        twords = row['target_words'].split('|')#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!#original target words
        #print(twords)
        #if train_flag:
        gs_word = row['gold_stds'].split()
        twords_gstd = [[tw, gs] for tw,gs in zip(twords, gs_word)]  ###('America',0.1)
        #temp = set([remove_noise(tgroup) for tgroup in twords]) #remove noise from every target group/word ######################################################################## TODO nu elimina noise-ul aici ca sa nu alterezi contextul

        eligible_words=[] # all processed words from the sentence (used to create lematized text)
        sent = ""
        twords_sentence_lem =[] #only processed words which are on 'word' col
        twords_sentence_unlem = [] #unlematized version of the lematized words
        groups_left = [] #group of words which won't be processed by context

        twg_lem=[]
        temp_twg_lem = []
        ######################################################## (procesat doar target words si gold_std) PANA AICI am extras toate grupurile de (word,gold_std) disponibile in tot datasetul din cele date la target words si am creat un set cu aceste target words

        #######################################################  (urmeaza procesarea propozitiei)
        twords_trace = twords.copy()
        for token in tokenized_sentence: #iau fiecare cuvant din propozitie
            ini_word = str(token) #save the unlematized word
            ################################################### (salvez in forma lemma lowercase toate cuvintele din propozitie care nu sunt stop words
            if not(token.is_stop) and not (any(char.isdigit() for char in str(token))) and not(str(token).isspace()) and len(str(token.lemma_))>1: #remove stop words and strings which contain digits and words of one letter
                token_lema = token.lemma_
                eligible_words.append(str(token_lema).lower()) #add every lemmatized word 

                ############################################### (salvez in temp_twg_lem cuvantul in forma lemma lowercase daca este target word, impreuna cu gold_std
                if ini_word in twords_trace: #if the unlematized word is in the unlematized word col
                    #print("*tt:",len(twords_trace)) 
                    twords_trace.remove(ini_word)
                    #print("**tt:",len(twords_trace))
                    
                    #print(twords_trace)
                    if train_flag:
                        #print(token_lema.lower())
                        temp_twg_lem.append([ float(row['gold_std']), ini_word, token_lema.lower(), word_gst(twords_gstd, ini_word)]) #create new matrix with the lemmatized word and its gold standard
                        #print("*",token_lema.lower())
                        upd_gs.append(word_gst(twords_gstd, ini_word))
                    else:
                        temp_twg_lem.append([row[0], ini_word, token_lema.lower(), word_gst(twords_gstd, ini_word)])
                    upd_tw.append(token_lema.lower())
                    twords_sentence_lem.append(token_lema.lower()) #then add the contextual lematized version to twords_proc
                    twords_sentence_unlem.append(str(ini_word))
            ################################################### (daca cuvantul din propozite este stop_word si se afla in target words, atunci salvez perechea in temp_twg_lem si in vocabular
            #                                                    SAU daca cuvantul
            elif token.is_stop and len(str(token.lemma_))>1:# and (str(token))[0].isupper(:
                token_lema = token.lemma_
                eligible_words.append(str(token_lema).lower()) #add every lemmatized word
                if ini_word in twords: #if the unlematized word is in the unlematized word col
                    #print("*tt:",len(twords_trace)) 
                    twords_trace.remove(ini_word)
                    #print("**tt:",len(twords_trace))
                    if train_flag:
                        #print(token_lema.lower())
                        temp_twg_lem.append([ float(row['gold_std']), ini_word, token_lema.lower(), word_gst(twords_gstd, ini_word)]) #*
                        #print("**",token_lema.lower())
                        upd_gs.append(word_gst(twords_gstd, ini_word))
                    else:
                        temp_twg_lem.append([row[0], ini_word, token_lema.lower(), word_gst(twords_gstd, ini_word)])
                    upd_tw.append(token_lema.lower())
                    twords_sentence_lem.append(token_lema.lower()) #then add the contextual lematized version to twords_proc
                    twords_sentence_unlem.append(str(ini_word))

        #process group of words that match 'word' column
        ################################################### (compar numarul de cuvinte adaugate in matrice cu cele din target words; ar trebui sa iasa in plus grupurile din target words

        if len(twords_sentence_unlem) != len(twords):
            groups_left = list(set(twords)^set(twords_sentence_unlem))
            '''
            print("tsu: ",twords_sentence_unlem)
            print("tw:  ",twords)
            print("gl:  ",groups_left)
            print("")
            '''
            
            for group in groups_left:
                #if group == '-':
                #    continue
                twords_trace.remove(str(group))
                lem_group=[]
                tok_group = nlp(group)
                for word in tok_group:
                    if str(word) == '-' or str(word) == '—':
                        continue
                    lem_group.append(str(word.lemma_).lower())
                if train_flag:
                    #print(lem_group)
                    temp_twg_lem.append([float(row['gold_std']), group, " ".join(lem_group), word_gst(twords_gstd, group)]) #add to the matrix the group of words################################ (adaug in matrice grupul de cuvinte cu cuvintele lematizate individual)
                    upd_gs.append(word_gst(twords_gstd, ini_word))
                else:
                    temp_twg_lem.append([row[0], group, " ".join(lem_group), word_gst(twords_gstd, group)])
                twords_sentence_lem.append(" ".join(lem_group)) #add the local lematized group to the lematized target words array
                upd_tw.append(lem_group)

        ################################################### (daca numarul de target words e diferit de numarul de elemente din matricea mea)
        twg_lem = temp_twg_lem
        elig_sent = " ".join(eligible_words) ###############creez propozitie cu cuvintele eligibile
        eligible_sentences.append(remove_noise(elig_sent)) #mai elimin noise-ul (redundant)
        twords_lem.append(twords_sentence_lem) ########adaug propozitia eligibila la array-ul general de cuvinte
        
 ########################################################################## 
        if(len(twords)!= len(twg_lem)): #for the case when a target word appears more than the appearances in the sentence
            '''print("twords:  ",twords)
            x=[el[-3] for el in twg_lem]
            print("twg_lem: ",x)
            print("rest:    ",twords_trace)'''
            
            for el in twords_trace:
                lem_group=[]
                tok_group = nlp(el)
                for word in tok_group:
                    if str(word) == '-' or str(word) == '—':
                        continue
                    lem_group.append(str(word.lemma_).lower())
                if len(el.split()) > 1:
                    if train_flag:
                        temp_twg_lem.append([float(row['gold_std']), str(el), " ".join(lem_group), word_gst(twords_gstd, el)]) #add to the matrix the group of words################################ (adaug in matrice grupul de cuvinte cu cuvintele lematizate individual)
                        upd_gs.append(word_gst(twords_gstd, el))
                    else:
                        temp_twg_lem.append([row[0], str(el), " ".join(lem_group), word_gst(twords_gstd, el)])
                    twords_sentence_lem.append(" ".join(lem_group)) #add the local lematized group to the lematized target words array
                else:
                    if train_flag:
                        temp_twg_lem.append([float(row['gold_std']), str(el), str(lem_group), word_gst(twords_gstd, el)]) #add to the matrix the group of words################################ (adaug in matrice grupul de cuvinte cu cuvintele lematizate individual)
                        upd_gs.append(word_gst(twords_gstd, el))
                    else:
                        temp_twg_lem.append([row[0], str(el), str(lem_group), word_gst(twords_gstd, el)])
                    twords_sentence_lem.append(str(lem_group)) #add the local lematized group to the lematized target words array
                upd_tw.append(str(lem_group))
  ######################################################################################                      
        twords_gstd_all.append(twg_lem)        ########adauga matricea la matricea generala de (target word, gold_std)
        
        input_file.at[idx, 'sentence'] = elig_sent
        joined_upd_tw = []
        for el in upd_tw:
            if type(el) == list:
                joined_upd_tw.append(" ".join(el))
            else:
                joined_upd_tw.append(el)
        input_file.at[idx, 'target_words'] = "|".join(map(str, joined_upd_tw)) #create a string from the list
        joined_upd_tw.clear()
        upd_tw.clear()
        upd_gs.clear()
        
    ### finished iterating through dataset
    
    ########################################
    eligible_text = " ".join(input_file['sentence'].values) #all the distinct sentences without punctuation, and strings with digits
    temp = flatten(twords_lem)
    twords_lem = temp

    words_elig_sent=" ".join(eligible_sentences).split()
    
    for el in twords_lem:
        if el not in words_elig_sent:
            if len(el.split()) == 1:
                words_elig_sent.append(el)
    eligible_text = " ".join(words_elig_sent)

    temp = flatten(twords_gstd_all)
    twords_gst_all = temp
    ########################################
    eligible_text = " ".join(input_file['sentence'].values) #all the distinct sentences without punctuation, and strings with digits
    temp = flatten(twords_lem)
    words_elig_sent=" ".join(eligible_sentences).split()


            
    for el in temp:
        if el not in words_elig_sent:
            if len(el.split()) == 1:
                words_elig_sent.append(el)
    eligible_text = " ".join(words_elig_sent)

    temp = flatten(twords_gstd_all)
    twords_gst_all = temp

    return flatten(twords_gstd_all), input_file, eligible_text

# Preprocess data

In [ ]:
!pip install gensim
!pip install requests-html

In [21]:
file = pd.read_csv('train_full.csv')
testfile = pd.read_csv('test.csv')

train_data = file.iloc[:11017]
test_data = file.iloc[11017:]

#Extract the sentence the total gold_std for every words labeled from every sentence, the labeled words and their gold standard
fabr_gstd = [0] * len(testfile.index)
testfile['gold_std'] = fabr_gstd

transformed_train = transform_df(file, train_flag = True)
transformed_official_test = transform_df(testfile, train_flag = False)

twords_gstd_all, train_file, eligible_text = extract_features(transformed_train, train_flag = True)
twords_gstd_official_test, new_testfile, eligible_text_official = extract_features(transformed_official_test, train_flag = False)
twords_gstd_all_test = twords_gstd_official_test

# Features

### 1) bigrams

In [ ]:
tw = [el[1] for el in twords_gstd_all] #all target words

two_g = []
two_g.clear()
for word in tw:
    two_g.append([ word[i:i+2] for i in range(len(word)-1) if len(word[i:i+2]) == 2 and " " not in word[i:i+2]])
    
bg_sent = [" ".join(el) for el in two_g]
bg_dict = dict(Counter(flatten(two_g)))
print(len(ng_dict))

763


### Word present in news dump - feature

In [22]:
import urllib.request,sys,time
from bs4 import BeautifulSoup
import requests
import pandas as pd

pagesToGet= 50
article_links = []
upperframe=[]  
for page in range(1,pagesToGet+1):
    print('processing page :', page)
    url = 'https://www.politifact.com/factchecks/list/?page='+str(page)
    print(url)
    
    #an exception might be thrown, so the code should be in a try-except block
    try:
        #use the browser to get the url. This is suspicious command that might blow up.
        page=requests.get(url)                             # this might throw an exception if something goes wrong.
    
    except Exception as e:                                   # this describes what to do if an exception is thrown
        error_type, error_obj, error_info = sys.exc_info()      # get the exception information
        print ('ERROR FOR LINK:',url)                          #print the link that cause the problem
        print (error_type, 'Line:', error_info.tb_lineno)     #print error info and line that threw the exception
        continue                                              #ignore this page. Abandon this and go back.
    time.sleep(2)   
    soup=BeautifulSoup(page.text,'html.parser')
    frame=[]
    links=soup.find_all('li',attrs={'class':'o-listicle__item'})
    print(len(links))
    
    for j in links:
        Statement = j.find("div",attrs={'class':'m-statement__quote'}).text.strip()
        #print(Statement)
        Link = "https://www.politifact.com"
        Link += j.find("div",attrs={'class':'m-statement__quote'}).find('a')['href'].strip()
        article_links.append(Link)

processing page : 1
https://www.politifact.com/factchecks/list/?page=1
30
processing page : 2
https://www.politifact.com/factchecks/list/?page=2
30
processing page : 3
https://www.politifact.com/factchecks/list/?page=3
30
processing page : 4
https://www.politifact.com/factchecks/list/?page=4
30
processing page : 5
https://www.politifact.com/factchecks/list/?page=5
30
processing page : 6
https://www.politifact.com/factchecks/list/?page=6
30
processing page : 7
https://www.politifact.com/factchecks/list/?page=7
30
processing page : 8
https://www.politifact.com/factchecks/list/?page=8
30
processing page : 9
https://www.politifact.com/factchecks/list/?page=9
30
processing page : 10
https://www.politifact.com/factchecks/list/?page=10
30
processing page : 11
https://www.politifact.com/factchecks/list/?page=11
30
processing page : 12
https://www.politifact.com/factchecks/list/?page=12
30
processing page : 13
https://www.politifact.com/factchecks/list/?page=13
30
processing page : 14
https://w

In [ ]:
import gensim
import nltk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import seaborn as sns
from newspaper import Article

topic_texts = []
preprocessed_files = []

articles = ['https://www.politifact.com/factchecks/2021/dec/22/facebook-posts/sealed-room-would-snake-plants-keep-you-alive-stud/','https://www.politifact.com/factchecks/2021/dec/22/facebook-posts/sealed-room-would-snake-plants-keep-you-alive-stud/']
c=0
for article_link in article_links:
    print(c)
    c+=1
    article = Article(url="%s" % article_link)
    article.download()
    article.parse()
    paragraphs = article.text.split("\n")
    #print(paragraphs)
    for i,el in enumerate(paragraphs):
        if "We rate this" in el:
            paragraphs.remove(el)
    par_list = "\n".join(paragraphs)
    pp_text = preprocess(par_list)  #stem every word and remove special characters                                                               
    txt = " ".join(pp_text)   #use the first 100 eligibile words from every wikipedia page
    topic_texts.append(txt.split())
temp = topic_texts.copy()
preprocessed_files.append(temp)       #save all preprocessed texts in preprocesed_files

In [ ]:
from collections import Counter
f = open("news_.txt", "r", encoding="utf-8")
all_articles = f.read().split("\n")
temp = [ article for article in all_articles if article!= ""]
all_articles = temp

all_articles_wlist = [article.split() for article in all_articles]
article_dictionary = dict(Counter(flatten(all_articles_wlist)))
#print(len(article_dictionary))
print(len(flatten(all_articles_wlist)))
print(len(article_dictionary))
top_dict = sorted(article_dictionary.items(), key=lambda x: x[1], reverse = True)
common_words = [key[0] for key in top_dict[:len(article_dictionary)//10]]
print(common_words)

In [ ]:
news_dump_presence_train = []
news_dump_presence_test = []

for el in tw:
    if el in common_words:
        news_dump_presence_train.append(1)
    else:
        news_dump_presence_train.append(0)
        
for el in tw_test:
    if el in common_words:
        news_dump_presence_test.append(1)
    else:
        news_dump_presence_test.append(0)
additional_ftrs_train.append(np.array(news_dump_presence_train))
additional_ftrs_test.append(np.array(news_dump_presence_test))

print(len(additional_ftrs_train))
print(len(additional_ftrs_test))

### Numeric feature

In [ ]:
additional_ftrs_train = []
additional_ftrs_test = []

alpha_train = []
alpha_test = []

for word in tw:
    if word.isalpha(): #contains only letters
        alpha_train.append(1)
    else:
        alpha_train.append(0)
for word in tw_test:
    if word.isalpha(): #contains only letters
        alpha_test.append(1)
    else:
        alpha_test.append(0)

additional_ftrs_train.append(np.array(alpha_train))
additional_ftrs_test.append(np.array(alpha_test))

print(sum(additional_ftrs_train))
print(len(additional_ftrs_test))
print(len(alpha_test))

### (X) word Length - feature

In [ ]:
character_count_train = [len(string) for string in tw]
character_count_test = [len(string) for string in tw_test]

additional_ftrs_train.append(np.array(character_count_train))
additional_ftrs_test.append(np.array(character_count_test))

print(len(additional_ftrs_train))
print(len(additional_ftrs_test))

### 2) sylabble count

In [ ]:
syl_count = []
for word in tw:
    syl_count.append(syllable_count(word))

print(len(syl_count))

11014


### 3) contains number


In [ ]:
alpha = []
for word in tw:
    if word.isalpha(): #contains only letters
        alpha.append(1)
    else:
        alpha.append(0)

11014
1810


### 4) Multi word expression

In [ ]:
mwe = []

for string in tw:
    if " " in string:             #is multi word expression
        mwe.append(1)
    else:
        mwe.append(0)
print(mwe)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 

### 5) number of characters

In [ ]:

character_count = [len(string) for string in tw]
print(character_count)

[6, 7, 5, 5, 8, 8, 4, 3, 3, 11, 6, 4, 6, 4, 10, 5, 8, 18, 14, 14, 14, 5, 11, 6, 7, 9, 20, 3, 7, 6, 5, 10, 11, 10, 8, 6, 11, 22, 10, 4, 7, 10, 7, 5, 4, 10, 5, 7, 10, 5, 10, 4, 8, 4, 10, 5, 5, 9, 9, 4, 9, 7, 9, 5, 17, 22, 8, 5, 6, 10, 4, 12, 5, 7, 7, 7, 7, 8, 6, 11, 10, 7, 7, 10, 18, 23, 24, 6, 7, 5, 4, 8, 14, 3, 3, 6, 3, 11, 7, 7, 5, 15, 12, 11, 7, 7, 3, 5, 5, 3, 5, 7, 8, 10, 5, 10, 3, 8, 4, 9, 12, 7, 24, 13, 9, 7, 10, 6, 4, 10, 6, 7, 5, 9, 5, 6, 8, 6, 6, 2, 7, 5, 4, 6, 9, 6, 15, 16, 22, 17, 7, 6, 8, 6, 8, 12, 6, 6, 6, 6, 5, 7, 7, 4, 6, 6, 5, 9, 4, 5, 12, 9, 22, 15, 21, 9, 6, 7, 6, 6, 6, 5, 6, 7, 4, 6, 6, 4, 8, 4, 6, 7, 6, 11, 5, 6, 35, 14, 10, 5, 7, 7, 9, 5, 8, 9, 7, 5, 4, 9, 4, 7, 4, 8, 15, 13, 10, 30, 16, 10, 8, 6, 7, 6, 6, 8, 9, 6, 7, 3, 5, 10, 6, 7, 6, 6, 9, 6, 8, 20, 13, 16, 15, 7, 4, 11, 10, 12, 7, 4, 6, 6, 6, 4, 5, 7, 43, 20, 11, 29, 13, 20, 7, 9, 5, 8, 11, 6, 8, 6, 7, 4, 9, 5, 3, 8, 7, 11, 5, 6, 8, 7, 11, 9, 7, 7, 7, 8, 5, 7, 9, 10, 6, 7, 24, 17, 7, 9, 9, 4, 9, 9, 4, 5, 9, 10, 

### 6)

In [ ]:
https://en.wikinews.org/wiki

In [ ]:
import urllib.request,sys,time
from bs4 import BeautifulSoup
import requests
import pandas as pd

pagesToGet= 100
article_links = []
upperframe=[]  
for page in range(1,pagesToGet+1):
    print('processing page :', page)
    url = 'https://www.politifact.com/factchecks/list/?page='+str(page)
    print(url)
    
    #an exception might be thrown, so the code should be in a try-except block
    try:
        #use the browser to get the url. This is suspicious command that might blow up.
        page=requests.get(url)                             # this might throw an exception if something goes wrong.
    
    except Exception as e:                                   # this describes what to do if an exception is thrown
        error_type, error_obj, error_info = sys.exc_info()      # get the exception information
        print ('ERROR FOR LINK:',url)                          #print the link that cause the problem
        print (error_type, 'Line:', error_info.tb_lineno)     #print error info and line that threw the exception
        continue                                              #ignore this page. Abandon this and go back.
    time.sleep(2)   
    soup=BeautifulSoup(page.text,'html.parser')
    frame=[]
    links=soup.find_all('li',attrs={'class':'o-listicle__item'})
    print(len(links))
    
    for j in links:
        Statement = j.find("div",attrs={'class':'m-statement__quote'}).text.strip()
        #print(Statement)
        Link = "https://www.politifact.com"
        Link += j.find("div",attrs={'class':'m-statement__quote'}).find('a')['href'].strip()
        article_links.append(Link)

import gensim
import nltk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import seaborn as sns
from newspaper import Article

topic_texts = []
preprocessed_files = []

articles = ['https://www.politifact.com/factchecks/2021/dec/22/facebook-posts/sealed-room-would-snake-plants-keep-you-alive-stud/','https://www.politifact.com/factchecks/2021/dec/22/facebook-posts/sealed-room-would-snake-plants-keep-you-alive-stud/']
c=0
for article_link in article_links:
    print(c)
    c+=1
    article = Article(url="%s" % article_link)
    article.download()
    article.parse()
    paragraphs = article.text.split("\n")
    #print(paragraphs)
    for i,el in enumerate(paragraphs):
        if "We rate this" in el:
            paragraphs.remove(el)
    par_list = "\n".join(paragraphs)
    pp_text = preprocess(par_list)  #stem every word and remove special characters                                                               
    txt = " ".join(pp_text)   #use the first 100 eligibile words from every wikipedia page
    topic_texts.append(txt.split())
temp = topic_texts.copy()
preprocessed_files.append(temp)       #save all preprocessed texts in preprocesed_files

processing page : 1
https://www.politifact.com/factchecks/list/?page=1
30
processing page : 2
https://www.politifact.com/factchecks/list/?page=2
30
processing page : 3
https://www.politifact.com/factchecks/list/?page=3
30
processing page : 4
https://www.politifact.com/factchecks/list/?page=4
30
processing page : 5
https://www.politifact.com/factchecks/list/?page=5
30
processing page : 6
https://www.politifact.com/factchecks/list/?page=6
30
processing page : 7
https://www.politifact.com/factchecks/list/?page=7
30
processing page : 8
https://www.politifact.com/factchecks/list/?page=8
30
processing page : 9
https://www.politifact.com/factchecks/list/?page=9
30
processing page : 10
https://www.politifact.com/factchecks/list/?page=10
30
processing page : 11
https://www.politifact.com/factchecks/list/?page=11
30
processing page : 12
https://www.politifact.com/factchecks/list/?page=12
30
processing page : 13
https://www.politifact.com/factchecks/list/?page=13
30
processing page : 14
https://w

In [ ]:
print(len(flatten(topic_texts)))
text_articles = [" ".join(el) for el in topic_texts]
print(len(text_articles))

with open("news_.txt", 'w', encoding="utf-8") as f:
    for text in text_articles:
        print(str(text),file=f)
f.close()

1092052
3000
